In [17]:
import xarray as xr
import zarr
import rechunker
import numpy as np
import pandas as pd

In [12]:
x = xr.DataArray(
    0.1 * np.arange(10),
    dims=["lat"],
    coords={"lat": np.arange(10)},
    name="sst",

)

In [13]:
x

<xarray.DataArray 'sst' (lat: 10)>
array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
Coordinates:
  * lat      (lat) int64 0 1 2 3 4 5 6 7 8 9

In [14]:
xs = xr.where(x.sst<0.5,np.nan,x,keep_attrs=True)

AttributeError: 'DataArray' object has no attribute 'sst'

In [10]:
xs

<xarray.DataArray 'sst' (lat: 10)>
array([nan, nan, nan, nan, nan, 0.5, 0.6, 0.7, 0.8, 0.9])
Coordinates:
  * lat      (lat) int64 0 1 2 3 4 5 6 7 8 9

<xarray.DataArray 'sst' (lat: 10)>
array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
Coordinates:
  * lat      (lat) int64 0 1 2 3 4 5 6 7 8 9

In [18]:
np.random.seed(0)
temperature = 15 + 8 * np.random.randn(2, 2, 3)
precipitation = 10 * np.random.rand(2, 2, 3)
lon = [[-99.83, -99.32], [-99.79, -99.23]]
lat = [[42.25, 42.21], [42.63, 42.59]]
time = pd.date_range("2014-09-06", periods=3)
reference_time = pd.Timestamp("2014-09-05")

In [55]:
ds = xr.Dataset(
    data_vars=dict(
        temperature=(["x", "y", "time"], temperature),
        precipitation=(["x", "y", "time"], precipitation),
        dum=(["x","y"],temperature[:,:,0]),
    ),
    coords=dict(
        lon=(["x", "y"], lon),
        lat=(["x", "y"], lat),
        time=time,
        reference_time=reference_time,
    ),
    attrs=dict(description="Weather related data."),
)
ds

<xarray.Dataset>
Dimensions:         (x: 2, y: 2, time: 3)
Coordinates:
    lon             (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat             (x, y) float64 42.25 42.21 42.63 42.59
  * time            (time) datetime64[ns] 2014-09-06 2014-09-07 2014-09-08
    reference_time  datetime64[ns] 2014-09-05
Dimensions without coordinates: x, y
Data variables:
    temperature     (x, y, time) float64 29.11 18.2 22.83 ... 18.28 16.15 26.63
    precipitation   (x, y, time) float64 5.68 9.256 0.7104 ... 7.992 4.615 7.805
    dum             (x, y) float64 29.11 32.93 22.6 18.28
Attributes:
    description:  Weather related data.

In [84]:
ds2 = ds.drop_vars("dum")
ds2

<xarray.Dataset>
Dimensions:         (x: 2, y: 2, time: 3)
Coordinates:
    lon             (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat             (x, y) float64 42.25 42.21 42.63 42.59
  * time            (time) datetime64[ns] 2014-09-06 2014-09-07 2014-09-08
    reference_time  datetime64[ns] 2014-09-05
Dimensions without coordinates: x, y
Data variables:
    temperature     (x, y, time) float64 29.11 18.2 22.83 ... 18.28 16.15 26.63
    precipitation   (x, y, time) float64 5.68 9.256 0.7104 ... 7.992 4.615 7.805
Attributes:
    description:  Weather related data.

In [86]:
ds2 = ds2.where(np.logical_and(ds.temperature>9,ds.temperature<20))
ds2

<xarray.Dataset>
Dimensions:         (x: 2, y: 2, time: 3)
Coordinates:
    lon             (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat             (x, y) float64 42.25 42.21 42.63 42.59
  * time            (time) datetime64[ns] 2014-09-06 2014-09-07 2014-09-08
    reference_time  datetime64[ns] 2014-09-05
Dimensions without coordinates: x, y
Data variables:
    temperature     (x, y, time) float64 nan 18.2 nan nan ... 18.28 16.15 nan
    precipitation   (x, y, time) float64 nan 9.256 nan nan ... 7.992 4.615 nan
Attributes:
    description:  Weather related data.

In [87]:
ds3=ds2.assign(dum=ds["dum"])
ds3

<xarray.Dataset>
Dimensions:         (x: 2, y: 2, time: 3)
Coordinates:
    lon             (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat             (x, y) float64 42.25 42.21 42.63 42.59
  * time            (time) datetime64[ns] 2014-09-06 2014-09-07 2014-09-08
    reference_time  datetime64[ns] 2014-09-05
Dimensions without coordinates: x, y
Data variables:
    temperature     (x, y, time) float64 nan 18.2 nan nan ... 18.28 16.15 nan
    precipitation   (x, y, time) float64 nan 9.256 nan nan ... 7.992 4.615 nan
    dum             (x, y) float64 29.11 32.93 22.6 18.28
Attributes:
    description:  Weather related data.

In [ ]:
ds.temperature

In [52]:
ds2.temperature

<xarray.DataArray 'temperature' (x: 2, y: 2, time: 3)>
array([[[        nan, 18.20125767,         nan],
        [        nan,         nan,         nan]],

       [[        nan, 13.78914233, 14.17424919],
        [18.28478802, 16.15234857,         nan]]])
Coordinates:
    lon             (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat             (x, y) float64 42.25 42.21 42.63 42.59
  * time            (time) datetime64[ns] 2014-09-06 2014-09-07 2014-09-08
    reference_time  datetime64[ns] 2014-09-05
Dimensions without coordinates: x, y

In [53]:
ds2.precipitation

<xarray.DataArray 'precipitation' (x: 2, y: 2, time: 3)>
array([[[       nan, 9.25596638,        nan],
        [       nan,        nan,        nan]],

       [[       nan, 8.70012148, 9.78618342],
        [7.99158564, 4.61479362,        nan]]])
Coordinates:
    lon             (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat             (x, y) float64 42.25 42.21 42.63 42.59
  * time            (time) datetime64[ns] 2014-09-06 2014-09-07 2014-09-08
    reference_time  datetime64[ns] 2014-09-05
Dimensions without coordinates: x, y

In [69]:
x = xr.Dataset(
    {
        "temperature_c": (
            ("lat", "lon"),
            20 * np.random.rand(4).reshape(2, 2),
        ),
        "precipitation": (("lat", "lon"), np.random.rand(4).reshape(2, 2)),
    },
    coords={"lat": [10, 20], "lon": [150, 160]},
)
x

<xarray.Dataset>
Dimensions:        (lat: 2, lon: 2)
Coordinates:
  * lat            (lat) int64 10 20
  * lon            (lon) int64 150 160
Data variables:
    temperature_c  (lat, lon) float64 2.365 12.8 2.867 18.89
    precipitation  (lat, lon) float64 0.5218 0.4147 0.2646 0.7742

In [71]:
x.assign(temperature_f=x["temperature_c"][:,0] * 9 / 5 + 32)

<xarray.Dataset>
Dimensions:        (lat: 2, lon: 2)
Coordinates:
  * lat            (lat) int64 10 20
  * lon            (lon) int64 150 160
Data variables:
    temperature_c  (lat, lon) float64 2.365 12.8 2.867 18.89
    precipitation  (lat, lon) float64 0.5218 0.4147 0.2646 0.7742
    temperature_f  (lat) float64 36.26 37.16

In [105]:
str1 = '2014-01-01T01:00:00'
time1 = np.datetime64(str1)#,dtype='datetime64[ns]')
time2 = time1+np.timedelta64(30, "D");
timea = np.arange('2014-01-01T01:00:00','2014-01-31T02:00:00',np.timedelta64(1, "h"),dtype='datetime64[ns]')
len(timea)

721

In [107]:
display(time1,time2)
display(type(time1),type(time2),type(timea))

numpy.datetime64('2014-01-01T01:00:00')

numpy.datetime64('2014-01-31T01:00:00')

numpy.datetime64

numpy.datetime64

numpy.ndarray